In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
df = pd.read_csv('train.csv')

In [2]:
n_features = df.shape[1]-1
n_examples = df.shape[0]
h1 = 500
h2 = 100
num_classes = 10
lr = 0.001
num_epochs = 2
batch_size = 140

In [3]:
print(df.shape)

(42000, 785)


In [4]:
df_train = df.sample(frac=0.75)
df_test = df.drop(df_train.index)

In [5]:
train_target = torch.tensor(df_train['label'].values.astype(np.float32))
train_features = torch.tensor(df_train.drop(['label'],axis=1).values.astype(np.float32)) 
train_tensor = torch.utils.data.TensorDataset(train_target,train_features) 
train_loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

In [6]:
test_target = torch.tensor(df_test['label'].values.astype(np.float32))
test_features = torch.tensor(df_test.drop(['label'],axis=1).values.astype(np.float32)) 
test_tensor = torch.utils.data.TensorDataset(test_target,test_features) 
test_loader = torch.utils.data.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = True)

In [7]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, h1) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(h1, h2)
        self.relu = nn.ReLU()
        self.l3 = nn.Linear(h2,num_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out

In [8]:
model = NeuralNet(n_features, h1, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)  

In [9]:
n_total_steps = len(train_loader)
l = []
i = 0
for epoch in range(2):
    for i, (labels,images) in enumerate(train_loader):  
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device).long()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        l.append(loss.item())
        i = i+1
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [10]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for labels,images in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the {len(df_test)} test images: {acc} %')

Accuracy of the network on the 10500 test images: 95.19047619047619 %
